# <center> Google Landmark Recognition</center>
# ![image](https://images.unsplash.com/photo-1526548583898-58820894ac9b?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=500&q=300)


Hi Everyone this is my first kernel, which gives a brief idea of using PCA and t-SNE to visualize the features extracted from images.

- **Please Upvote if you find this kernel interesting**
- Suggestions and feedbacks are most welcome

### About the Competition

The Third Landmark Recognition competition! This year, we have worked to set this up as a code competition and collected a new set of test images.

Have you ever gone through your vacation photos and asked yourself: What was the name of that place I visited in 5 years ago? or Who created this monument I saw in France? Landmark recognition can help! This technology can predict landmark labels directly from image pixels, to help people better understand and organize their photo collections. This competition challenges Kagglers to build models that recognize the correct landmark (if any) in a dataset of challenging test images.

**EVALUATION**

Submissions are evaluated using Global Average Precision (GAP) at k, where k=1. This metric is also known as micro Average Precision (μAP) It works as follows:

For each test image, you will predict one landmark label and a corresponding confidence score. The evaluation treats each prediction as an individual data point in a long list of predictions (sorted in descending order by confidence scores), and computes the Average Precision based on this list.

If a submission has N predictions (label/confidence pairs) sorted in descending order by their confidence scores, then the Global Average Precision is computed as:

**$$GAP=\frac{1}{M}\sum_{i=1}^{N}P(i)rel(i)$$**

where:

- N is the total number of predictions returned by the system, across all queries
- M is the total number of queries with at least one landmark from the training set visible in it (note that some queries may not depict landmarks)
- P(i) is the precision at rank i
- rel(i) denotes the relevance of prediciton i: it’s 1 if the i-th prediction is correct, and 0 otherwise

**TIMELINE**
- September 22, 2020 - Entry deadline. You must accept the competition rules before this date in order to compete.

- September 22, 2020 - Team Merger deadline. This is the last day participants may join or merge teams.

- September 29, 2020 - Final submission deadline.

- October 6, 2020 - Paper Submission deadline (winners only).**

* Winners will be required to provide a detailed description of their method in order to claim their prize (minimum of 2 pages double-column).

All deadlines are at 11:59 PM UTC on the corresponding day unless otherwise noted. The competition organizers reserve the right to update the contest timeline if they deem it necessary.

In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
import time

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from tqdm import tqdm_notebook

import torch,torchvision
from albumentations import *
from albumentations.pytorch import ToTensor

from torchvision import models
import warnings
warnings.filterwarnings("ignore")
sns.set()

In [ ]:
train_df = pd.read_csv('../input/landmark-recognition-2020/train.csv')
train_df.head()

**Trian Data**
- Total Number of Training Data: **1580470**
- Total Number of Classes in Trian data: **81313**

In [ ]:
train_df['landmark_id'].value_counts().values.max(),train_df['landmark_id'].value_counts().values.min()

In [ ]:
plt.figure(figsize=(10,5))
sns.distplot(train_df['landmark_id'].value_counts().values,bins=10)
plt.show()

**From the above graph it is clear that train data is unequally distributed across all the classes**
- Max Number of Images avalible for a class - 6272 (class name: 138982)
- Min Number of Images available for a class - 2 (class name: 197219)

In [ ]:
value_counts_df = pd.DataFrame(train_df['landmark_id'].value_counts())
value_counts_df.index.name='Class_name'
value_counts_sorted = value_counts_df.sort_values('Class_name')
value_counts_sorted.reset_index('Class_name',inplace=True)
plt.figure(figsize=(20,5))
plt.plot(value_counts_sorted.Class_name.values,value_counts_sorted['landmark_id'].values,linestyle='--',
         marker='*',color = 'red')
plt.xlabel('Landmark_id',fontsize=12)
plt.ylabel('Number of Images',fontsize=12)
plt.show()

In [ ]:
df = pd.DataFrame(value_counts_df['landmark_id'].value_counts())
plt.figure(figsize=(20,5))
plt.subplot(121)
plt.plot(df['landmark_id'].values,df.index,linestyle='-',
         marker='.',color = '#52B9FB',label='Complete Data')
plt.plot(df[df['landmark_id']>1]['landmark_id'].values,df[df['landmark_id']>1].index,linestyle='-',
         marker='.',color = '#EE0D73',label='class count>1')
plt.plot(df[df['landmark_id']>25]['landmark_id'].values,df[df['landmark_id']>25].index,linestyle='-',
         marker='.',color = '#397D09',label='class count>25')
plt.plot(df[df['landmark_id']>50]['landmark_id'].values,df[df['landmark_id']>50].index,linestyle='-',
         marker='.',color = '#C58A17',label='class count>50')
plt.plot(df[df['landmark_id']>100]['landmark_id'].values,df[df['landmark_id']>100].index,linestyle='-',
         marker='.',color = '#F2625E',label='class count>100')
plt.ylabel('Images Count',fontsize=12)
plt.xlabel('Number of Landmark_id(classes)',fontsize=12)
plt.legend()

plt.subplot(122)
x_label = ['2-5','5-10','10-50','50-100','100-250','250-500','>500']
y_label = [df[(df.index <= 5)]['landmark_id'].sum(),df[(df.index > 5) & (df.index<=10)]['landmark_id'].sum(),
          df[(df.index > 10) & (df.index<=50)]['landmark_id'].sum(),df[(df.index > 50) & (df.index<=100)]['landmark_id'].sum(),
          df[(df.index > 100) & (df.index<=250)]['landmark_id'].sum(),df[(df.index > 250) & (df.index<=500)]['landmark_id'].sum(),
          df[(df.index > 500)]['landmark_id'].sum()]

plt.barh(x_label,y_label)
for i, v in enumerate(y_label):
    plt.text(v + 3, i-0.1, str(v),fontsize=8)
    
plt.ylabel('Images Count',fontsize=12)
plt.xlabel('Number of Landmark_id(classes)',fontsize=12)
plt.show()

- **92%** of the existing classes has less than 50 image count

## Images

- Since the Dataset is huge images are stored in three subset of folders where the first 3 digits in "train_df.id" represents the folders. So we can access the data as shown below

In [ ]:
train_path = '../input/landmark-recognition-2020/train/'
# 5 images from 5 different class
np.random.seed(27)
classes = np.random.choice(train_df['landmark_id'].unique(),5,replace=False)
for cls in classes:
    image_names = np.random.choice(train_df[train_df['landmark_id'] == cls]['id'],5,replace=False)
    
    c=1
    plt.figure(figsize=(25,5))
    for image_name in image_names:
        image_label = cls
        image = cv2.imread(os.path.join(train_path,image_name[0],image_name[1],image_name[2],image_name+'.jpg'))
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        plt.subplot(1,5,c)
        plt.title('Image_name: '+image_name + '\n\nLabel: '+str(image_label))
        plt.axis('off')
        plt.imshow(image)
        c+=1
    plt.show()

**Observation from the above grids of images**
- **Label 100601**,the statue is constant in all images
- **Label - 55401**, has different views, I wonder how a model is going to train to detect the structure when a boat is placed infront.
- **Label 172124**, the features of the image inside the building is completly different from outside.(Images 1,2 and 4 are similar whereas 3 and 5 are completely different)  

## Classes With Images >500 ( 51 Classes)

In [ ]:
class Load_Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.image_paths = df['id']
        self.labels = df['landmark_id']
        self.default_transform = Compose([
            Normalize((0.485, 0.456, 0.406),
                                 (0.229, 0.224, 0.225),always_apply=True),
            Resize(224,224),
            ToTensor()
        ])
        
    def __len__(self):
        return self.image_paths.shape[0]
    
    def __getitem__(self,i):
        image_name = self.image_paths[i]
        img_path = os.path.join('../input/landmark-recognition-2020/train/',
                                                     image_name[0],image_name[1],image_name[2],image_name+'.jpg')
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image = self.default_transform(image=image)['image']
        mean = torch.mean(image)
        std = torch.std(image)
        label = torch.tensor(self.labels[i])
        return image,label,mean,std

In [ ]:
# class_500 = value_counts_df[value_counts_df['landmark_id']>500].index.values
# class_500_df = train_df[train_df['landmark_id'].isin(class_500)]
# class_500_df.reset_index(inplace=True,drop=True)
# class_500_dataset = Load_Dataset(class_500_df)
# class_500_dataset_loader = torch.utils.data.DataLoader(class_500_dataset,batch_size=64)

## Extracting Feature Descriptors Using Resnet-50

In [ ]:
# model = models.resnet50(pretrained=True)
# model = torch.nn.Sequential(*(list(model.children())[:-1]))
# device='cuda'
# model.to(device)

# output_descriptor = np.zeros((1,2048))
# output_label = np.zeros((1))
# output_means = np.zeros((1))
# output_stds = np.zeros((1))

# with torch.no_grad():
#     for _,(images,labels,means,stds) in tqdm_notebook(enumerate(class_500_dataset_loader)):
#         images,labels,means,stds = images.to(device),labels.to(device),means.to(device),stds.to(device)
#         model.eval()
#         predictions = model(images)
#         output_descriptor = np.concatenate((output_descriptor,predictions.cpu().numpy().squeeze()),0)
#         output_label = np.concatenate((output_label,labels.cpu().numpy()))
#         output_means = np.concatenate((output_means,means.cpu().numpy()))
#         output_stds = np.concatenate((output_stds,stds.cpu().numpy()))

        
# output_descriptor = output_descriptor[1:]
# output_label = output_label[1:]
# output_means = output_means[1:]
# output_stds = output_stds[1:]

# output_500 = pd.concat([pd.DataFrame(output_descriptor),pd.DataFrame(output_label),pd.DataFrame(output_means),pd.DataFrame(output_stds)],1)
# col_names = []
# for idx,col in enumerate(output_500.columns):
#     if idx < 2048:
#         col_names.append('f_'+str(col))
#     elif idx == 2048:
#         col_names.append('label')
#     elif idx == 2049:
#         col_names.append('mean')
#     else:
#         col_names.append('std')     
# output_500.columns = col_names
# output_500.to_csv('resnet_50_features.csv',index=False)

In [ ]:
output_500 = pd.read_csv('../input/gld-resnet-50-features/resnet_50_features.csv')

In [ ]:
plt.figure(figsize=(30,10))
plt.suptitle('Mean and Standard Deviation of Images', fontsize=14,fontweight='bold')
plt.subplot(211)
sns.boxplot(x=output_500.label.astype(int),y=output_500['mean'].astype(np.float64))
plt.xticks(rotation=90)
plt.xlabel('Class')

plt.subplot(212)
sns.boxplot(x=output_500.label.astype(int),y=output_500['std'].astype(np.float64))
plt.xticks(rotation=90)
plt.xlabel('Class')
plt.show()

In [ ]:
output_500['label'] = output_500['label'].astype(int)
output_500['mapped_label'] = output_500['label'].astype('category')
output_500['mapped_label'] = output_500['mapped_label'].cat.codes
output_500.head(2)

## Train Validation Split(80-20)

In [ ]:
train,valid = train_test_split(output_500,stratify=output_500['mapped_label'],
                              test_size=0.20)
x_train = train.iloc[:,:-4]
x_valid = valid.iloc[:,:-4]
y_train = train['mapped_label']
y_valid= valid['mapped_label']
x_train.shape,x_valid.shape

In [ ]:
def plot_scatter(x, colors,true_label,train=True):
    # choose a color palette with seaborn.
    num_classes = len(np.unique(colors))
    palette = np.array(sns.color_palette("Set1", num_classes))

    # create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40, c=palette[colors.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # add the labels for each digit corresponding to the label
    txts = []

    for i in range(num_classes):

        # Position of each label at median of data points.

        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=14)
#         txt = ax.text(xtext, ytext, str(true_label[i]), fontsize=14)
#         txt.set_path_effects([
#             PathEffects.Stroke(linewidth=1, foreground="w"),
#             PathEffects.Normal()])
#         txts.append(txt)
    if train:
        plt.title("Trian Data")
    else:
        plt.title("Valid Data")
    plt.show()

#     return f, ax, sc, txt

## Principal Component Analysis(PCA)--> Linear Dimensionality Reduction

In [ ]:
st_time = time.time()
pca = PCA(n_components=5)
pca_result_train = pca.fit_transform(x_train)
pca_result_test = pca.fit_transform(x_valid)

print('PCA done; Time take {} seconds'.format(time.time()-st_time))
print('Variance: {}'.format(pca.explained_variance_ratio_))
print('Sum of variance in data by first top five components: {:.2f}%'.format(100*(pca.explained_variance_ratio_.sum())))

##PCA df
pca_tr = pd.DataFrame(columns=['pca1','pca2'])
pca_te = pd.DataFrame(columns=['pca1','pca2'])

pca_tr['pca1'] = pca_result_train[:,0]
pca_tr['pca2'] = pca_result_train[:,1]

pca_te['pca1'] = pca_result_test[:,0]
pca_te['pca2'] = pca_result_test[:,1]

plot_scatter(pca_tr[['pca1','pca2']].values,y_train,train['label'].values)
plot_scatter(pca_te[['pca1','pca2']].values,y_valid,valid['label'].values,False)


In [ ]:
print('True Label for 31st class as shown in above plot is {}'.format(str(train[train['mapped_label'] == 31]['label'].values[0])))

You can PCA was not able to cluster for all the classes, this may be due to the extracted features using resnet-50 (trained on Imagenet dataset). PCA might be able to give better clustering if we could try extracting features using model trained on similar dataset.

## t- Distributed Stochastic Neighbout (t-SNE)-->Non-linear Dimensionality Reduction Technique

T-sne took almost **2 Hours 14 Minutes to run** in CPU mode so i have saved the features as numpy as array and saved in "GLDR RESNET 50 FEATURES" dataset and made public  
* Here is the link to access GLDR RESNET 50 FEATURES Dataset - https://www.kaggle.com/ramjib/gld-resnet-50-features
* DATASET includes only features for the 51 classes for which number of images where more than 500. (Will try to extract for all the images and update the dataset asap)

In [ ]:
# st_time = time.time()
# t_sne = TSNE(random_state=2020)
# t_sne_tr = t_sne.fit_transform(x_train)
# t_sne_va = t_sne.fit_transform(x_valid)

# np.save('t_sne_tr.npy',t_sne_tr)
# np.save('t_sne_va.npy',t_sne_va)

# print('TNSE done; Time take {} seconds'.format(time.time()-st_time))


# plot_scatter(t_sne_tr, y_train,train['label'].values)
# plot_scatter(t_sne_va, y_valid,valid['label'].values,False)

image = cv2.cvtColor(cv2.imread('../input/gld-resnet-50-features/tsne_train.png'), cv2.COLOR_BGR2RGB)
plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.axis('off')
plt.show()

t-SNE was able to cluster the classes more clearly than PCA as we can observe

### Using PCA features for t-sne

In [ ]:
st_time = time.time()
pca = PCA(n_components=100)
pca_result_train = pca.fit_transform(x_train)
pca_result_test = pca.fit_transform(x_valid)

print('PCA done; Time take {} seconds'.format(time.time()-st_time))
# print('Variance: {}'.format(pca.explained_variance_ratio_))
print('Sum of variance in data by first top five components: {:.2f}%'.format(100*(pca.explained_variance_ratio_.sum())))

In [ ]:
st_time = time.time()
t_sne = TSNE(random_state=2020)
t_sne_tr = t_sne.fit_transform(pca_result_train)
t_sne_va = t_sne.fit_transform(pca_result_test)
print('TNSE done; Time take {} seconds'.format(time.time()-st_time))

In [ ]:
plot_scatter(t_sne_tr, y_train,train['label'].values)
plot_scatter(t_sne_va, y_valid,valid['label'].values,False)

In [ ]:
t_sne_tr.shape

Will try to do more visualizations in coming days. Thank You!!

## References:
- https://www.datacamp.com/community/tutorials/introduction-t-sne